In [1]:
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

#### ISING MODEL

In [113]:
class IsingModel:
    def __init__(self, lr=0.0005):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        self.gaussian_scale = 0.01
        
    
    def fit_activations(self, activations_list):
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(activations_list)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # Calculate the mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
        # Calculate the correlation
        self.data_correlation = self.calculate_corr(self.data_activations_matrix)
        
        #Create H and J matrices
        self.create_H_J()
        
         
    def fit_m_c(self, m, C):
        self.data_mean = m
        self.data_correlation = C
    
    def create_H_J(self):
        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=self.gaussian_scale, size=self.data_correlation.shape)
        lind = np.tril_indices_from(self.J)
        self.J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=self.gaussian_scale, size=self.num_spins)
        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0001
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())
            normalized_dict = {sigma: prob / Z for sigma, prob in prob_dict.items()}
            # Compute the statistics
            model_mean, model_correlation = self.get_statistics(normalized_dict)


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            # Take the step
            oldJ = copy.deepcopy(self.J)
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Calculate the variation of J for the termination condition
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
            
            epoch += 1
            
        print('Final epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
        print(model_correlation, 'C Model')
        print(self.data_correlation, 'C Data')

        print(model_mean, 'Average Model')
        print(self.data_mean, 'Average Data')
        print()

        print(self.H, 'H')
        print(self.J, 'J')

    # Calculate the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    def get_delta_energy(self, i):
        #delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])
        
        delta_energy= 2 * (self.H[i]*self.sigma[i] \
                           + self.sigma[i] * np.dot(self.J[i, :], self.sigma) \
                           + self.sigma[i] * np.dot(self.J[:, i], self.sigma))
        
        return delta_energy
    
    def metropolis_step(self):
        # Randomly select a spin (now it can only select 0)
        spin_to_flip = np.random.randint(self.num_spins)

        #delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])
        delta_energy = self.get_delta_energy(spin_to_flip)

        p = math.exp(-delta_energy)
        r = random.random()

        #print(delta_energy, r, p)
        if delta_energy <= 0 or r<=p:
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]

    def metropolis_simulation(self, max_steps, initial_burn_in_len=0, inter_burn_in=0):
        # Generate a random initial spin configuration
        self.sigma = random.choices(self.spin_values, k=self.num_spins)        
        
        # Create a dictionary to count the frequencies
        samples_dict = {}
        for i in range(0, max_steps):
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i >= initial_burn_in_len and (i%inter_burn_in)==0:
                if not tuple(copy.deepcopy(self.sigma)) in samples_dict: # If the state is new, create an entry
                    samples_dict[tuple(copy.deepcopy(self.sigma))] = 1
                else: # If the state already appeared, add one
                    samples_dict[tuple(copy.deepcopy(self.sigma))] += 1
                
            
        # Normalize the probability
        sum_states = sum(samples_dict.values())
        normalized_dict = {k: v / sum_states for k, v in samples_dict.items()}
                                                 
        return normalized_dict

    def get_statistics(self, states_dict):
        # Returns the magnetization and correlation from a dictionary of activations {sigma, prob}
        magnetization =  np.zeros(self.num_spins)
        corr_matrix = np.zeros((self.num_spins,self.num_spins))
        # Calculate the expectation
        for sigma, prob in states_dict.items():
            magnetization += np.dot(sigma,prob)
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]*prob
        return magnetization, corr_matrix
    
    # Calculates the correlation of a list of activations
    def calculate_corr(self, activations_matrix):
        num_samples_data = activations_matrix.shape[0]
        
        corr_matrix = np.zeros((self.num_spins, self.num_spins))
        for sigma in activations_matrix:
            # Calculate \sigma_i * \sigma_j
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]
        corr_matrix = corr_matrix / float(num_samples_data)
        
        return corr_matrix
    
    def train_metropolis(self, max_epochs = 300, simulation_len = 100, burn_in_len=0, inter_burn_in=0):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0003
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:

            samples_dict = self.metropolis_simulation(simulation_len, burn_in_len, inter_burn_in)
        
            # Calculate the data expectations        
            model_mean, model_correlation = self.get_statistics(samples_dict)
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            oldJ = copy.deepcopy(self.J)
            # Take the step
            self.J = self.J + stepJ
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of J and H for the termination condition
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%50 == 0 or epoch == max_epochs-1:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
                
            epoch +=1
            
        print('Final epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
        print(model_correlation, 'C Model')
        print(self.data_correlation, 'C Data')

        print(model_mean, 'Average Model')
        print(self.data_mean, 'Average Data')
        print()

        
        print(self.H, 'H')
        print(self.J, 'J')
        
    

In [114]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = IsingModel(lr=0.01)
s_ising.fit_activations(activations_list)

In [115]:
s_ising.train(10000)

Epoch 100 TotalParamVariation 0.00769781
[[ 0.         -0.31108665  0.21083466 -0.12227821]
 [ 0.          0.         -0.21231623 -0.11728099]
 [ 0.          0.          0.          0.00225436]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.21012599 -0.21577771  0.33590111 -0.0031277 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 200 TotalParamVariation 0.00377666
[[ 0.         -0.39707385  0.25497925 -0.15862384]
 [ 0.          0.         -0.2553139  -0.1573121 ]
 [ 0.          0.          0.          0.00127369]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.25444781 -0.25695109  0.43351222 -0.00122212] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 300 TotalParamVariation 0.00269328

Epoch 1900 TotalParamVariation 0.00039467
[[ 0.00000000e+00 -4.87000856e-01  2.50080445e-01 -2.36920950e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50047991e-01 -2.36920994e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.24956310e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50047891e-01 -2.50080550e-01  4.99971846e-01 -1.25845255e-05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2000 TotalParamVariation 0.00037511
[[ 0.00000000e+00 -4.87636615e-01  2.50071420e-01 -2.37564642e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50044261e-01 -2.37564669e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.04659474e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

Epoch 3800 TotalParamVariation 0.00019802
[[ 0.00000000e+00 -4.93435504e-01  2.50016356e-01 -2.43416001e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50015145e-01 -2.43416001e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.71135527e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50015145e-01 -2.50016356e-01  4.99994018e-01 -4.71154389e-07] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 3900 TotalParamVariation 0.00019296
[[ 0.00000000e+00 -4.93602560e-01  2.50015456e-01 -2.43584057e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50014434e-01 -2.43584057e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.97807308e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

Epoch 5500 TotalParamVariation 0.00013686
[[ 0.00000000e+00 -4.95454973e-01  2.50007500e-01 -2.45445747e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50007431e-01 -2.45445747e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.71610992e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50007431e-01 -2.50007500e-01  4.99997260e-01 -2.71611108e-08] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 5600 TotalParamVariation 0.00013442
[[ 0.00000000e+00 -4.95535822e-01  2.50007229e-01 -2.45526926e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50007170e-01 -2.45526926e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.29910200e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

Epoch 7400 TotalParamVariation 0.00010171
[[ 0.00000000e+00 -4.96618991e-01  2.50004107e-01 -2.46613923e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50004104e-01 -2.46613923e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.16031308e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50004104e-01 -2.50004107e-01  4.99998522e-01 -1.16031317e-09] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 7500 TotalParamVariation 0.00010035
[[ 0.00000000e+00 -4.96663979e-01  2.50003997e-01 -2.46659047e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50003994e-01 -2.46659047e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.83534580e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

In [ ]:
s_ising.train_metropolis( max_epochs = 10000, simulation_len = 500, burn_in_len=25,  inter_burn_in=5)